In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
import pandas as pd
import numpy as np
import scipy as sp

import datetime
from collections import Counter
from sklearn.externals.joblib import Parallel, delayed
%matplotlib inline

In [2]:
#Helper Functions
def SelectDataColumns():
    column_dictionary = {}
    column_dictionary['Admissions'] = ['ADMISSION_TYPE','ADMISSION_LOCATION','INSURANCE','ETHNICITY',
                                       'HOSPITAL_EXPIRE_FLAG','ADMITTIME','DISCHTIME']
    column_dictionary['Patient'] = ['GENDER','DOB']
    return column_dictionary

def LagTime(x):
    try:
        return float((x['ICU_Time']-x['Admit_Time']).total_seconds())/60/60
    except:
        return(np.nan)

def TimeOfDay(x):
    if((x.hour>7) & (x.hour<19)):
        return('Day')
    return 'Night'
    
def CreateColumnHeaders():
    header_list = ['pat_id','hadm_id']
    #Admissions
    header_list.extend(['Admit_Type','Admit_LOC','Insurance','Ethnicity','Expire','Admit_Time','Discharge_Time'])
    #Patient
    header_list.extend(['Gender','DOB'])
    #ICU Stay
    header_list.extend(['Admit_Unit','LOS_ICU','ICU_Time'])
    #Transfer
    header_list.extend(['NoICU','Transfer_Flag'])
    #Diagnosis
    header_list.append('Primary_Diag')
    #DRG
    header_list.append('DRG')
    return header_list

def BuildPatientData(pat_id,hadm_id):
    patient_data = [pat_id,hadm_id]
    #Admissions Table Data
    admissions_temp = admissions_table[admissions_table['HADM_ID']==hadm_id].to_dict('records')[0]
    for col in column_dictionary['Admissions']:
        patient_data.append(admissions_temp.get(col,np.nan))  
        
    #Patient Table Data
    patient_temp = patient_table[patient_table['SUBJECT_ID']==pat_id].to_dict('records')[0]
    for col in column_dictionary['Patient']:
        patient_data.append(patient_temp.get(col,np.nan))
        
    #ICU_Stay Table
    icu_stay_temp = icu_stay_table[icu_stay_table['HADM_ID']==hadm_id]
    if(icu_stay_temp.shape[0]==0):
        patient_data.extend([np.nan]*2)
        patient_data.append(pd.Timestamp.min)
    else:
        patient_data.append(icu_stay_temp['FIRST_CAREUNIT'].values[0])
        patient_data.append(icu_stay_temp['LOS'].values[0])
        patient_data.append(icu_stay_temp.to_dict('records')[0]['INTIME'])

    #Transfers Table
    transfers_temp = transfers_table[transfers_table['HADM_ID']==hadm_id]
    patient_data.append(1*(any((transfers_temp['EVENTTYPE']=='admit') & (pd.isnull(transfers_temp['PREV_CAREUNIT'])) & (pd.isnull(transfers_temp['CURR_CAREUNIT'])))))
    ##Were they transfered
    patient_data.append(1*(len(transfers_temp['CURR_CAREUNIT'].value_counts().values)>1))
    
    
    #Diagnosis (Primary)
    diagnosis_temp = diagnosis_table[(diagnosis_table['HADM_ID']==hadm_id) & (diagnosis_table['SEQ_NUM']==1)]
    if(diagnosis_temp.shape[0]==0):
        patient_data.append(np.nan)
    else:
        patient_data.append(diagnosis_temp['ICD9_CODE'].values[0])
    
    #DRG
    drg_temp = drg_table[(drg_table['HADM_ID']==hadm_id)]
    if(drg_temp.shape[0]==0):
        patient_data.append(np.nan)
    else:
        patient_data.append(drg_temp['DRG_CODE'].values[0])
    return patient_data

In [3]:
#Load Relevant Tables
admissions_table = pd.read_csv('../MIMICIII/ADMISSIONS.csv')
patient_table = pd.read_csv('../MIMICIII/PATIENTS.csv')
transfers_table = pd.read_csv('../MIMICIII/TRANSFERS.csv')
icu_stay_table = pd.read_csv('../MIMICIII/ICUSTAYS.csv')
diagnosis_table = pd.read_csv('../MIMICIII/DIAGNOSES_ICD.csv')
drg_table = pd.read_csv('../MIMICIII/DRGCODES.csv')
services_table = pd.read_csv('../MIMICIII/SERVICES.csv')

In [4]:
#Clean Data
#Admissions / convert to datetime
admissions_table['ADMITTIME'] = pd.to_datetime(admissions_table['ADMITTIME'], format='%Y-%m-%d %H:%M:%S')
admissions_table['DISCHTIME'] = pd.to_datetime(admissions_table['DISCHTIME'], format='%Y-%m-%d %H:%M:%S')

#Patient DOB
patient_table['DOB'] = pd.to_datetime(patient_table['DOB'], format='%Y-%m-%d %H:%M:%S')

#ICU STAY
icu_stay_table['INTIME'] = pd.to_datetime(icu_stay_table['INTIME'], format='%Y-%m-%d %H:%M:%S')

In [5]:
#Check for those with a single ICU Stay (help to prevent confounding from frequent fliers or readmissions)
g = admissions_table.groupby('SUBJECT_ID')['HADM_ID'].size()
print('Total Admissions',admissions_table.shape[0],' By: ',len(admissions_table['SUBJECT_ID'].unique()),"Patients")
print("Patient's with Two or more admissions",g[g>1].shape)
print("Patient's with A single or more admissions",g[g==1].shape) #This will be the data we work with

filtered_data = admissions_table[admissions_table['SUBJECT_ID'].isin(g[g==1].reset_index()['SUBJECT_ID'].values)]

column_dictionary = SelectDataColumns()
data = Parallel(n_jobs=-1)(delayed(BuildPatientData)(row[1]['SUBJECT_ID'],row[1]['HADM_ID']) for row in filtered_data.iterrows())

Total Admissions 58976  By:  46520 Patients
Patient's with Two or more admissions (7537,)
Patient's with A single or more admissions (38983,)


In [6]:
pat_data = pd.DataFrame(data,columns=CreateColumnHeaders())

#Derive Columns
pat_data['Age'] = pat_data.apply(lambda x: float((x['Admit_Time']-x['DOB']).total_seconds())/60/60/24/365,axis=1)
del pat_data['DOB']

pat_data['Hospital_LOS'] = pat_data.apply(lambda x: float((x['Discharge_Time']-x['Admit_Time']).total_seconds())/60/60/24,axis=1) 
del pat_data['Discharge_Time']

pat_data['Lag_Time'] = pat_data.apply(lambda x: LagTime(x),axis=1)
pat_data['TOD'] = pat_data['ICU_Time'].apply(lambda x: TimeOfDay(x))
pat_data['DOW'] = pat_data['ICU_Time'].apply(lambda x: x.dayofweek)

pat_data = pat_data[pat_data['Lag_Time']>0]

(614, 21)


In [7]:
#pat_data.to_pickle('pat_data.pkl')

In [8]:
pat_data.head()

,pat_id,hadm_id,Admit_Type,Admit_LOC,Insurance,Ethnicity,Expire,Admit_Time,Gender,Admit_Unit,...,ICU_Time,NoICU,Transfer_Flag,Primary_Diag,DRG,Age,Hospital_LOS,Lag_Time,TOD,DOW
0,22,165315,EMERGENCY,EMERGENCY ROOM ADMIT,Private,WHITE,0,2196-04-09 12:26:00,F,MICU,...,2196-04-09 12:27:00,0,0,9678,450.0,64.971282,1.144444,0.016667,Day,5
1,24,161859,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,Private,WHITE,0,2139-06-06 16:14:00,M,CCU,...,2139-06-06 16:15:36,0,0,41041,526.0,39.042949,2.856944,0.026667,Day,5
2,25,129635,EMERGENCY,EMERGENCY ROOM ADMIT,Private,WHITE,0,2160-11-02 02:06:00,M,CCU,...,2160-11-02 03:16:23,1,0,41071,526.0,58.989281,3.534028,1.173056,Night,6
3,26,197661,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,Medicare,UNKNOWN/NOT SPECIFIED,0,2126-05-06 15:16:00,M,CCU,...,2126-05-07 09:52:30,1,0,99604,515.0,72.053798,6.988889,18.608333,Day,1
4,27,134931,NEWBORN,PHYS REFERRAL/NORMAL DELI,Private,WHITE,0,2191-11-30 22:16:00,F,NICU,...,2191-11-30 22:21:15,0,1,V3000,390.0,0.002542,2.686806,0.087500,Night,2


In [9]:
services_table.head()

,ROW_ID,SUBJECT_ID,HADM_ID,TRANSFERTIME,PREV_SERVICE,CURR_SERVICE
0,758,471,135879,2122-07-22 14:07:27,TSURG,MED
1,759,471,135879,2122-07-26 18:31:49,MED,TSURG
2,760,472,173064,2172-09-28 19:22:15,NaN,CMED
3,761,473,129194,2201-01-09 20:16:45,NaN,NB
4,762,474,194246,2181-03-23 08:24:41,NaN,NB
